<a href="https://colab.research.google.com/github/LC1332/Chat-Haruhi-Suzumiya/blob/main/notebook/%E4%BB%8EJsonl%E6%95%B0%E6%8D%AE%E6%A0%BC%E5%BC%8F%E5%BC%80%E5%A7%8B%E7%BB%84%E7%BB%87%E6%95%B0%E6%8D%AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ChatHaruhi的数据生成

这个脚本考虑使用jsonl进行生成

从texts和system prompt到jsonl格式的代码见

https://github.com/LC1332/Chat-Haruhi-Suzumiya/blob/main/characters/novel_collecting/chatbot%E7%9A%84embedding%E6%8A%BD%E5%8F%96%E5%92%8Cjsonl%E7%94%9F%E6%88%90.ipynb





In [1]:
!pip -q install openai tiktoken langchain chromadb datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.4/502.4 kB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

训练的参数设定

In [2]:
K_SEARCH = 5
MAX_LEN_STORY = 1000 #这个是按照token算的
MAX_LEN_HISTORY = 1200 # count with token

In [3]:
import os
import openai
key = "key is not necessary here"
key_bytes = key.encode()
os.environ["OPENAI_API_KEY"] = key_bytes.decode('utf-8')

这里我们假设input里面有很多jsonl

这里你需要自己把自己的jsonl格式数据放进来

In [9]:
!ls /content/input

女贤者.jsonl  安莉娜.jsonl  艾希2.jsonl  艾希.jsonl


In [10]:
fname2role_name = {
    "女贤者":"女贤者",
    "安莉娜":"安莉娜",
    "艾希2":"艾希",
    "艾希":"艾希",
}
input_path = "/content/input"

In [5]:
%cd /content
!rm -rf /content/Haruhi-2-Dev
!git clone https://github.com/LC1332/Haruhi-2-Dev
%cd /content/Haruhi-2-Dev

/content
Cloning into 'Haruhi-2-Dev'...
remote: Enumerating objects: 946, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (140/140), done.
remote: Total 946 (delta 172), reused 171 (delta 113), pack-reused 693
Receiving objects: 100% (946/946), 106.26 MiB | 46.96 MiB/s, done.
Resolving deltas: 100% (502/502), done.
/content/Haruhi-2-Dev


In [6]:
!ls

ChatHaruhi  data  LICENSE  notebook  Readme.md


In [7]:
from ChatHaruhi import ChatHaruhi

from ChatHaruhi.ChromaDB import ChromaDB

class ChatHaruhiTrain(ChatHaruhi):

    def build_story_db_from_vec( self, texts, vecs ):
        self.db = ChromaDB()
        self.stories = texts
        self.db.init_from_docs( vecs, texts)

    def add_story_with_expire(self, query, expire_story):
        if self.db is None:
            print("No vec DB！")
            return

        query_vec = self.embedding(query)
        stories = self.db.search(query_vec, self.k_search)

        story_string = self.story_prefix_prompt + self.dialogue_divide_token

        sum_story_token = self.tokenizer(story_string)

        for story in stories:
            if expire_story.strip() == story.strip():
                continue

            story_token = self.tokenizer(story.strip()) + self.tokenizer(self.dialogue_divide_token)
            if sum_story_token + story_token > self.max_len_story:
                break
            else:
                sum_story_token += story_token
                story_string += story.strip() + self.dialogue_divide_token

        self.llm.user_message(story_string)

    def generate_prompt(self, query, history_str, expire_story ):
        # 这里修改下其他超参，不规范删了
        # self.k_search = 5
        # self.max_len_story = 1500
        # self.max_len_history = 1200
        self.story_prefix_prompt = "\nClassic scenes for the role are as follows:"

        self.llm.initialize_message()

        self.llm.system_message(self.system_prompt)

        self.add_story_with_expire(query, expire_story)

        # self.add_history(history)
        history_message = self.dialogue_divide_token + history_str
        self.llm.user_message(history)

        self.llm.user_message(query)

        # self.llm.user_message(target)

        return self.llm.messages

    def add_history(self, history_list):

        if len(history_list) == 0:
            return

        sum_history_token = 0
        flag = 0
        for history in history_list:
            current_count = 0
            if history is not None:
                current_count += self.tokenizer(history)

            sum_history_token += current_count
            if sum_history_token > self.max_len_history:
                break
            else:
                flag += 1

        if flag == 0:
            print('warning! no history added. the last dialogue is too long.')

        # 是否添加历史前缀，
        history_message = ""
        for history in history_list[-flag:]:
            history_message += history
        self.llm.user_message(history_message)

In [8]:
def extract_tuples_from_story(role_name, story):
    story_tuples = []

    lines = story.split('\n')

    n = len(lines)

    datas = []

    for i in range(1, n):
        line = lines[i]
        if line.startswith(role_name):
            query = lines[i-1]
            target = line
            history_str = ""
            for j in range(0, i-1):
                history_str += lines[j] + "\n"
            data = (query, target, history_str, story)
            datas.append(data)

    return datas

对RoleLLM的数据进行组织

In [11]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [22]:
from datasets import load_dataset
from tqdm import tqdm

save_datas = []

for fname in fname2role_name:
    role_name = fname2role_name[fname]
    fname_jsonl = os.path.join(input_path, fname + ".jsonl")

    # 这里embedding要使用jsonl抽取对应的
    chatbot = ChatHaruhiTrain( role_from_jsonl = fname_jsonl, \
                      llm = 'openai')

    stories = chatbot.stories

    chatbot.k_search = K_SEARCH
    chatbot.max_len_story = MAX_LEN_STORY
    chatbot.max_len_history = MAX_LEN_HISTORY

    all_tuples = []

    for story in stories:
        # print(story)
        tuples = extract_tuples_from_story(role_name, story )

        all_tuples += tuples

    for query, target, history, story in all_tuples:
        messages = chatbot.generate_prompt(query, history, story)
        # print(prompt)

        prompt = ""

        for msg in messages:
            if isinstance(msg, HumanMessage):
                prompt += msg.content
            elif isinstance(msg, AIMessage):
                prompt += msg.content
            elif isinstance(msg, SystemMessage):
                prompt += msg.content

        save_data = {
            'context': prompt,
            'target': target
        }

        save_datas.append(save_data)

正在下载Luotuo-Bert


config.json:   0%|          | 0.00/35.9k [00:00<?, ?B/s]

models.py:   0%|          | 0.00/21.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/silk-road/luotuo-bert-medium:
- models.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

Luotuo-Bert下载完毕


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [18]:
print(len(save_datas))

215


save_datas是一个list of dict
save_name = "/content/ChatHaruhi_rolellm_fusion.jsonl"

帮我用utf-8编码，jsonl格式，保存save_datas中的数据
ensure_ascii = False

In [21]:
import json

save_name = "/content/ChatHaruhi_Waifu_extended.jsonl"

with open(save_name, 'w', encoding='utf8') as f:
    for data in save_datas:
        json.dump(data, f, ensure_ascii=False)
        f.write('\n')